# Adversarial Training on Pointer Generator
## Introduction
    The beginning of the introduction. 

## Table Of Contents:
* [Load Data & Initialize Model](#load-initialize)
* [Example Generation](#example-generation)
* [Train Pointer Generator](#train-global-1)
    * [Without Coverage](#train-global-1-sub-1)
        * [Generate Tokens](#gen-global-1-sub-1)
        * [Rouge Evaluation](#rouge-global-1-sub-1)
    * [With Coverage](#train-global-1-sub-2)
        * [Generate Tokens](#gen-global-1-sub-2)
        * [Rouge Evaluation](#rouge-global-1-sub-2)
* [Train Generative Adversarial Network](#train-global-2)
    * [Pretrain Discriminator](#train-global-2-sub-1)
        * [Generate Tokens](#gen-global-2-sub-1)
        * [Rouge Evaluation](#rouge-global-2-sub-1)
    * [Adversarial Training](#train-global-2-sub-2)
        * [Generate Tokens](#gen-global-2-sub-2)
        * [Rouge Evaluation](#rouge-global-2-sub-2)
* [Analysis & Conclusion](#analysis-conclusion)
* [Limitations & Future Work](#limit-future)


## Load Data & Initialize Model <a class="anchor" id="load-initialize"></a>

In [ ]:
import numpy as np
from data import Data
from model import SummaryModel
import argparse

import tensorflow as tf

tf.compat.v1.disable_eager_execution()
tf.compat.v1.logging.set_verbosity('ERROR')

parser = argparse.ArgumentParser(description = 'Train/Test summarization model', formatter_class = argparse.ArgumentDefaultsHelpFormatter)

# Import Setting
parser.add_argument("--doc_file", type = str, default = './data/doc.p', help = 'path to document file')
parser.add_argument("--vocab_file", type = str, default = './data/vocab.p', help = 'path to vocabulary file')
parser.add_argument("--emb_file", type = str, default = './data/emb.p', help = 'path to embedding file')
parser.add_argument("--src_time", type = int, default = 200, help = 'maximal # of time steps in source text')
parser.add_argument("--sum_time", type = int, default = 50, help = 'maximal # of time steps in summary')
parser.add_argument("--max_oov_bucket", type = int, default = 280, help = 'maximal # of out-of-vocabulary word in one summary')
parser.add_argument("--train_ratio", type = float, default = 0.8, help = 'ratio of training data')
parser.add_argument("--seed", type = int, default = 888, help = 'seed for spliting data')

# Saving Setting
parser.add_argument("--log", type = str, default = './log/', help = 'logging directory')
parser.add_argument("--save", type = str, default = './model/', help = 'model saving directory')
parser.add_argument("--checkpoint", type = str, help = 'path to checkpoint point')
parser.add_argument("--autosearch", type = bool, default = False, help = "[NOT AVAILABLE] Set 'True' if searching for latest checkpoint")
parser.add_argument("--save_interval", type = int, default = 1900, help = "Save interval for training")

# Hyperparameter Setting
parser.add_argument("--batch_size", type = int, default = 16, help = 'number of samples in one batch')
parser.add_argument("--gen_lr", type = float, default = 1e-3, help = 'learning rate for generator')
parser.add_argument("--dis_lr", type = float, default = 1e-3, help = 'learning rate for discriminator')
parser.add_argument("--cov_weight", type = float, default = 1e-3, help = 'learning rate for coverage')

params = vars(parser.parse_args([]))

params['load_pretrain'] = True
# 1900 no coverage
# 
params['checkpoint'] = './model/pointer_cov_supervised-1900' # Uncomment when requiring reloading model

model = SummaryModel(**params)
data = Data(**params)


## Example Generation <a class="anchor" id="example-generation"></a>

In [ ]:
train_data = data.get_next_epoch()
test_data = data.get_next_epoch_test()
src, ref, gen, tokens, scores, attens, gt_attens = None, None, None, None, None, None, None
for feed_dict in train_data:
    real, fake, real_len, fake_len = model.sess.run(
        [model.real_reward, model.fake_reward, model.sum_len, model.tokens_len], feed_dict=feed_dict)
    print(np.mean(real[1, 0:int(real_len[1])]))
    print(np.mean(fake[1, 0:int(fake_len[1])]))
    break

for feed_dict in test_data:
    tokens, scores, attens = model.beam_search(feed_dict)
    src, ref, gen = data.id2word(feed_dict, tokens)
    gt_attens = model.sess.run(model.atten_dist, feed_dict = feed_dict)
#     print(src, ref, gen, gt_attens)
    x = 0
    print ("".join(src[x]).replace("(OOV)",""), end = '\n\n')
    print ("".join(ref[x]).replace("(OOV)",""), end = '\n\n')
    print (gen)
#     for i in range(len(src)):
#         print ("".join(gen[x][i]))
    break

## Train Pointer Generator<a class="anchor" id="train-global-1"></a>
### Train without coverage<a class="anchor" id="train-global-1-sub-1"></a>

In [ ]:
train_max_epoch = 1
print (f'Start from step {model.sess.run(model.gen_global_step)}')
for i in range(train_max_epoch):
    print (f'Train Epoch {i}')
    train_data = data.get_next_epoch()
    model.train_one_epoch(train_data, data.n_train_batch, coverage_on = False)

#### Generate tokens <a class="anchor" id="gen-global-1-sub-1"></a>

In [ ]:
def generate_top_k_tokens(coverage, top_k=1):
    test_data = data.get_next_epoch_test()
    src = [[] for i in range(top_k)]
    ref = [[] for i in range(top_k)]
    gen = [[] for i in range(top_k)]
    for feed_dict in test_data:
        tokens, scores, attens = model.beam_search(feed_dict, coverage_on = coverage, top_k = top_k)
        if top_k == 1:
            tokens = [tokens]
            scores = [scores]
        for i in range(top_k):
            src[i], ref[i], gen[i] = data.id2word(feed_dict, tokens[i])
#         feed_dict['coverage_on:0'] = coverage
#         gt_attens = model.sess.run(model.atten_dist, feed_dict = feed_dict)
        break
    return src, ref, gen, scores

def clean_generated_tokens(src, ref, gen):
    src_str_list = ["".join(src[0][i]).replace("(OOV)", "") for i in range(len(src[0]))]
    ref_str_list = ["".join(ref[0][0]).replace("(OOV)", "") for i in range(len(ref[0]))]
    gen_str_list = []
    for i in range(len(test1_gen)):
        gen_str_list.append(["".join(gen[i][j]).replace("(OOV)","") for j in range(len(gen[i]))])
    return src_str_list, ref_str_list, gen_str_list

def print_cleaned_tokens(src, ref, gen, scores):
    for i in range(len(gen)):
        print ("Batch: " + str(i))
        for j in range(len(gen[i])):
            print("\tGeneration: " + str(j))
            print ("\t\tAbstract: " + src[j][:min(95, len(src[j]))])
            print ("\t\tTitle: "+ ref[j])
            print("\t\tGenerated: " + gen[i][j])
            print("\t\tScore: " + str(scores[i][j]))

In [ ]:
test1_src, test1_ref, test1_gen, test1_scores = generate_top_k_tokens(coverage=False, top_k=1)
test1_src_list, test1_ref_list, test1_gen_list = clean_generated_tokens(test1_src, test1_ref, test1_gen)
print_cleaned_tokens(test1_src_list, test1_ref_list, test1_gen_list, test1_scores)

#### Rouge Evaluation<a class="anchor" id="rouge-global-1-sub-1"></a>

In [ ]:
from rouge import Rouge
from tqdm import tqdm_notebook
rouge = Rouge()

def generate_evaluation_sets(coverage):
    refs = []
    gens = []
    cnt = 0
    test_n = min(data.n_test_batch, 50)
    test_data = data.get_next_epoch_test()
    for feed_dict in tqdm_notebook(test_data, total = test_n):
        tokens, scores, attens = model.beam_search(feed_dict, coverage_on = coverage)
        # sample_tokens = model.sess.run(model.tokens, feed_dict = feed_dict)
        src, ref, gen = data.id2word(feed_dict, tokens)
        for i in range(len(ref)):
            refs.append(" ".join(ref[i][:-1]))
            gens.append(" ".join(gen[i][:-1]))
        cnt += 1
        if cnt > test_n:
            break
    new_gens = []
    new_refs = []
    for i in range(len(gens)):
        if not (gens[i] == ""):
            new_gens.append(gens[i])
            new_refs.append(refs[i])
    return new_refs, new_gens


def rouge_evaluation(ref, gen):
    rouge_score = rouge.get_scores(gen, ref)
    r1, r2, rl = 0., 0., 0.
    for score in rouge_score:
        r1 = r1 + score['rouge-1']['f']
        r2 = r2 + score['rouge-2']['f']
        rl = rl + score['rouge-l']['f']
    r1 /= len(rouge_score)
    r2 /= len(rouge_score)
    rl /= len(rouge_score)
    print (r1, r2, rl)
    return r1, r2, rl

In [ ]:
test1_eval_refs, test1_eval_gens = generate_evaluation_sets(coverage=False)
test1_r1, test1_r2, test1_rl = rouge_evaluation(test1_eval_refs, test1_eval_gens)

### Train with coverage<a class="anchor" id="train-global-1-sub-2"></a>

In [ ]:
train_max_epoch = 1
print (f'Start from step {model.sess.run(model.gen_global_step)}')
for i in range(train_max_epoch):
    print (f'Train Epoch {i}')
    train_data = data.get_next_epoch()
    model.train_one_epoch(train_data, data.n_train_batch, coverage_on = True, model_name = 'with_coverage')

#### Generate tokens<a class="anchor" id="gen-global-1-sub-2"></a>

In [ ]:
test2_src, test2_ref, test2_gen, test2_scores = generate_top_k_tokens(coverage=True, top_k=1)
test2_src_list, test2_ref_list, test2_gen_list = clean_generated_tokens(test2_src, test2_ref, test2_gen)
print_cleaned_tokens(test2_src_list, test2_ref_list, test2_gen_list, test2_scores)

#### Rouge Evaluation<a class="anchor" id="rouge-global-1-sub-2"></a>

In [ ]:
test2_eval_refs, test2_eval_gens = generate_evaluation_sets(coverage=True)
test2_r1, test2_r2, test2_rl = rouge_evaluation(test2_eval_refs, test2_eval_gens)

## Train GAN<a class="anchor" id="train-global-2"></a>
### Pretrain Discriminator<a class="anchor" id="train-global-2-sub-1"></a>

In [ ]:
train_max_epoch = 1
print (f'Start from step {model.sess.run(model.gen_global_step_2)}')
for i in range(train_max_epoch):
    print (f'Train Epoch {i}')
    train_data = data.get_next_epoch()
    model.train_one_epoch_pre_dis(train_data, data.n_train_batch, coverage_on = True)

#### Generate tokens<a class="anchor" id="gen-global-2-sub-1"></a>

In [ ]:
test3_src, test3_ref, test3_gen, test3_scores = generate_top_k_tokens(coverage=True, top_k=1)
test3_src_list, test3_ref_list, test3_gen_list = clean_generated_tokens(test3_src, test3_ref, test3_gen)
print_cleaned_tokens(test3_src_list, test3_ref_list, test3_gen_list, test3_scores)

#### Rouge Evaluation<a class="anchor" id="rouge-global-2-sub-1"></a>

In [ ]:
test3_eval_refs, test3_eval_gens = generate_evaluation_sets(coverage=True)
test3_r1, test3_r2, test3_rl = rouge_evaluation(test3_eval_refs, test3_eval_gens)

### Adversarial Training<a class="anchor" id="train-global-2-sub-2"></a>

In [ ]:
train_max_epoch = 12
print (f'Start from step {model.sess.run(model.gen_global_step_2)}')
for i in range(train_max_epoch):
    print (f'Train Epoch {i}')
    train_data = data.get_next_epoch()
    model.train_one_epoch_unsup(train_data, data.n_train_batch, coverage_on = True)

#### Generate Tokens<a class="anchor" id="gen-global-2-sub-2"></a>

In [ ]:
test4_src, test4_ref, test4_gen, test4_scores = generate_top_k_tokens(coverage=True, top_k=1)
test4_src_list, test4_ref_list, test4_gen_list = clean_generated_tokens(test4_src, test4_ref, test4_gen)
print_cleaned_tokens(test4_src_list, test4_ref_list, test4_gen_list, test4_scores)

#### Rouge Evaluation<a class="anchor" id="rouge-global-2-sub-2"></a>

In [ ]:
test4_eval_refs, test4_eval_gens = generate_evaluation_sets(coverage=True)
test4_r1, test4_r2, test4_rl = rouge_evaluation(test4_eval_refs, test4_eval_gens)

In [ ]:
test_data = data.get_next_epoch_test()
src, ref, gen, tokens, scores, attens, gt_attens = None, None, None, None, None, None, None
for feed_dict in train_data:
    real, fake, real_len, fake_len = model.sess.run(
        [model.real_reward, model.fake_reward, model.sum_len, model.tokens_len], feed_dict=feed_dict)
    print(np.mean(real[1, 0:int(real_len[1])]))
    print(np.mean(fake[1, 0:int(fake_len[1])]))
    break

## Analysis & Conclusion<a class="anchor" id="analysis-conclusion"></a>

## Limitations & Future Work<a class="anchor" id="limit-future"></a>